In [64]:
import numpy as np
from scipy import io

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms, utils, datasets
from torch.utils.data import Dataset, DataLoader, TensorDataset

In [39]:
p1_dataset = io.loadmat('p1_dataset.mat')

In [71]:
# label : 1 (p1)
p1 = p1_dataset['p1_noise']
print(p1.shape)
p1_re = np.real(p1)
p1_im = np.imag(p1)
X_p1 = np.stack((p1_re,p1_im),axis=1)
y_p1 = np.ones(len(p1),dtype=int)
print(y_p1.shape)
# label : 0 (noise)
n = p1_dataset['noise_data']
print(n.shape)
n_re = np.real(n)
n_im = np.imag(n)
X_re = np.stack((p1_re,p1_im),axis=1)
y_re = np.zeros(len(n),dtype=int)
print(y_re.shape)


(8400, 2048)
(8400,)
(8400, 2048)
(8400,)


In [72]:
X = np.vstack((X_p1,X_re))
y = np.concatenate((y_p1,y_re),axis=None)
print(X.shape)
print(y.shape)

(16800, 2, 2048)
(16800,)


In [79]:
from torch.utils.data.dataset import random_split

x_tensor = torch.from_numpy(X).float()
y_tensor = torch.from_numpy(y).float()

dataset = TensorDataset(x_tensor,y_tensor)
dataset_size = len(dataset)
train_split = 0.7
train_size = int(train_split * dataset_size)
test_size = dataset_size - train_size

train_dataset, val_dataset = random_split(dataset, [train_size,test_size])
print(len(train_dataset), len(val_dataset))
# train_loader = DataLoader(dataset=train_dataset,batch_size=16)

11760 5040


In [ ]:
train_loader = DataLoader(dataset=train_dataset,
                        batch_size=batch_size,
                        shuffle=True, num_workers=2)


test_loader = DataLoader(dataset=val_dataset,
                        batch_size=batch_size,
                        shuffle=True, num_workers=2)

In [ ]:
# Device configuration
device = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')
print(device)
# Hyper-parameter
learning_rate = 0.001
batch_size = 400
num_epochs = 200

In [ ]:
class Binary_Classifier(nn.Module): 
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=3)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=3)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(720, 1024)
        self.fc2 = nn.Linear(1024, 2)

    def forward(self, x):
        x = torch.unsqueeze(x, dim=1) 
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(x.shape[0],-1)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return x

In [ ]:
LSTM_model = LSTM(input_size, hidden_size, layer_num,dropout, num_classes, sequence_length,device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(LSTM_model.parameters(), lr = learning_rate)

In [ ]:
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0, path=PATH, trace_func=print):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print            
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [ ]:
def train_model(model, batch_size, patience, n_epochs, train_loader , test_loader, PATH, device):

    train_losses = []
    valid_losses = []
    avg_train_losses = []
    avg_valid_losses = []
    early_stopping = EarlyStopping(patience= patience, verbose=True, path=PATH+'checkpoint.pt')

    for epoch in range(1, n_epochs + 1):
        
        ###################
        # train the model #
        ###################
        model.train()
        for batch_idx, (data, targets) in enumerate(train_loader):
            #data = F.normalize(data, p=2)
            data = data.to(device)
            targets = targets.to(device)
            #print(data.shape)
            
            # Pytorch Dataloader data output shape : Batch x 1 x Data shape
            
            # Required data shape for RNN : Batch x Sequence Lengh x Data shape
            optimizer.zero_grad()

            scores = model(data)

            loss = criterion(scores, torch.argmax(targets, dim=1))
            
            loss.backward()

            optimizer.step()
            
            train_losses.append(loss.item())

        ######################    
        # validate the model #
        ######################
        model.eval() # prep model for evaluation
        for data , target in test_loader :
            data = data.to(device)
            target = target.to(device)
            # forward pass: 입력된 값을 모델로 전달하여 예측 출력 계산
            output = model(data)
            # calculate the loss
            loss = criterion(output, torch.argmax(target, dim=1))
            # record validation loss
            valid_losses.append(loss.item())

        # print 학습/검증 statistics
        # epoch당 평균 loss 계산
        train_loss = np.average(train_losses)
        valid_loss = np.average(valid_losses)
        avg_train_losses.append(train_loss)
        avg_valid_losses.append(valid_loss)

        epoch_len = len(str(n_epochs))


        print_msg = (f'[{epoch:>{epoch_len}}/{n_epochs:>{epoch_len}}] ' +
                     f'train_loss: {train_loss:.5f} ' +
                     f'valid_loss: {valid_loss:.5f}')

        print(print_msg)

        # clear lists to track next epoch
        train_losses = []
        valid_losses = []

        # early_stopping는 validation loss가 감소하였는지 확인이 필요하며,
        # 만약 감소하였을경우 현제 모델을 checkpoint로 만든다.
        early_stopping(valid_loss, model)

        if early_stopping.early_stop:
            print("Early stopping")
            break

   # best model이 저장되어있는 last checkpoint를 로드한다.
    model.load_state_dict(torch.load(PATH + 'checkpoint.pt'))

    return  model, avg_train_losses, avg_valid_losses

In [ ]:
patience = 20
model, train_loss, valid_loss = train_model(LSTM_model, batch_size, patience, num_epochs, train_loader, test_loader, PATH, device)

In [ ]:
# 훈련이 진행되는 과정에 따라 loss를 시각화
fig = plt.figure(figsize=(10,8))
plt.plot(range(1,len(train_loss)+1),train_loss, label='Training Loss')
plt.plot(range(1,len(valid_loss)+1),valid_loss,label='Validation Loss')

# validation loss의 최저값 지점을 찾기
minposs = valid_loss.index(min(valid_loss))+1
plt.axvline(minposs, linestyle='--', color='r',label='Early Stopping Checkpoint')

plt.xlabel('epochs')
plt.ylabel('loss')
#plt.ylim(1,3) # 일정한 scale
#plt.xlim(0, len(train_loss)+1) # 일정한 scale
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()
fig.savefig(PATH + 'loss_plot.png', bbox_inches = 'tight')